# 🎬 Tech Challenge Fase 4 - Análise de Vídeo com IA

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mavsousa30/techchallengefase4/blob/main/notebooks/Colab_Video_Analysis.ipynb)

## 📋 O que este notebook faz:
1. ✅ Detecta rostos em vídeos
2. ✅ Classifica emoções faciais
3. ✅ Reconhece atividades humanas
4. ✅ Detecta anomalias
5. ✅ Gera relatórios automáticos

## ⚡ Ative a GPU!
**Runtime → Change runtime type → GPU (T4)**

## 1️⃣ Configuração Inicial

In [ ]:
# Verificar GPU
!nvidia-smi -L

# Verificar recursos
import psutil
print(f"\n💾 RAM Total: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"💾 RAM Disponível: {psutil.virtual_memory().available / (1024**3):.1f} GB")
print(f"🖥️  CPU Cores: {psutil.cpu_count()}")

In [ ]:
# Clone o repositório
!git clone https://github.com/Mavsousa30/techchallengefase4.git
%cd techchallengefase4

In [ ]:
# Verificar estrutura do projeto
!ls -la
print("\n✅ Arquivos do projeto:")
!ls -la src/

In [ ]:
# Instalar dependências (mais rápido no Colab)
!pip install -q opencv-python numpy pandas deepface torch torchvision tqdm pydantic rich moviepy

# Adicionar diretório do projeto ao Python path
import sys
import os

# Garantir que o diretório do projeto está no path
if '/content/techchallengefase4' not in sys.path:
    sys.path.insert(0, '/content/techchallengefase4')

print("✅ Dependências instaladas!")
print("✅ Python path configurado!")

## 2️⃣ Upload de Vídeo

In [ ]:
from google.colab import files
import os

# Upload do vídeo
print("📤 Faça upload do seu vídeo:")
uploaded = files.upload()

# Pegar o nome do arquivo
video_filename = list(uploaded.keys())[0]
print(f"\n✅ Vídeo carregado: {video_filename}")
print(f"📊 Tamanho: {len(uploaded[video_filename]) / (1024**2):.2f} MB")

## 3️⃣ Processamento do Vídeo

In [ ]:
# Importar módulos do projeto
import sys
import os

# Garantir que estamos no diretório correto
if not os.path.exists('src'):
    print("⚠️ Mudando para o diretório do projeto...")
    os.chdir('/content/techchallengefase4')

# Adicionar ao path se necessário
if '/content/techchallengefase4' not in sys.path:
    sys.path.insert(0, '/content/techchallengefase4')

print("📂 Diretório atual:", os.getcwd())
print("📦 Verificando módulos...")

try:
    from src.pipeline.inference import InferencePipeline
    from src.metrics.reporter import Reporter
    import json
    from IPython.display import Video, Markdown, JSON
    
    print("✅ Todos os módulos importados com sucesso!")
    print("🎬 Pronto para processar vídeo!\n")
except ImportError as e:
    print(f"❌ Erro ao importar módulos: {e}")
    print("\n🔍 Verificando estrutura do projeto:")
    !ls -la src/
    print("\n💡 Solução: Certifique-se que executou todas as células anteriores")
    raise

In [ ]:
# Criar diretórios necessários
import os

directories = ['outputs', 'outputs/logs', 'outputs/frames']
for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"✅ Diretório criado/verificado: {directory}")

In [ ]:
# Criar e executar pipeline
pipeline = InferencePipeline(
    video_path=video_filename,
    output_video_path="outputs/annotated_video.mp4",
    save_preview=True,
    face_backend="opencv",  # Mais rápido para começar
    emotion_backend="deepface"
)

# Processar (com GPU será MUITO mais rápido!)
summary = pipeline.run()

print("\n✅ Processamento concluído!")

## 4️⃣ Visualizar Resultados

In [ ]:
# Mostrar métricas principais
print("📊 RESUMO DO PROCESSAMENTO")
print("=" * 50)
print(f"📹 Vídeo: {summary['video_path']}")
print(f"🎞️  Total de Frames: {summary['frames_total']:,}")
print(f"⏱️  Duração: {summary['duration_seconds']:.2f}s")
print(f"🎯 FPS: {summary['fps']:.2f}")
print(f"\n👤 FACES")
print(f"  Total detectadas: {summary['faces_stats']['total']}")
print(f"  Média por frame: {summary['faces_stats']['mean_per_frame']:.2f}")
print(f"  Máximo em um frame: {summary['faces_stats']['max_in_frame']}")
print(f"\n😊 EMOÇÕES")
for emotion, count in sorted(summary['emotions_distribution'].items(), key=lambda x: x[1], reverse=True):
    print(f"  {emotion}: {count}")
print(f"\n🎭 ATIVIDADES")
print(f"  Total de eventos: {len(summary['activities_timeline'])}")
print(f"\n⚠️  ANOMALIAS")
print(f"  Total: {summary['anomalies_total']}")
for severity, count in summary['anomalies_by_severity'].items():
    print(f"  {severity}: {count}")

In [ ]:
# Visualizar vídeo anotado
print("\n🎬 Vídeo com Anotações:")
Video("outputs/annotated_video.mp4", width=800)

## 5️⃣ Gerar e Baixar Relatórios

In [ ]:
# Gerar relatórios
reporter = Reporter()

# Salvar JSON
reporter.save_metrics(summary, "outputs/metrics.json")
print("✅ metrics.json gerado")

# Salvar Markdown
reporter.generate_markdown_report(summary, "outputs/report.md")
print("✅ report.md gerado")

# Visualizar relatório markdown
with open("outputs/report.md", "r") as f:
    Markdown(f.read())

In [ ]:
# Baixar todos os arquivos
from google.colab import files
import zipfile
import os

# Criar ZIP com todos os resultados
zip_filename = "video_analysis_results.zip"

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write("outputs/annotated_video.mp4", "annotated_video.mp4")
    zipf.write("outputs/metrics.json", "metrics.json")
    zipf.write("outputs/report.md", "report.md")

print(f"📦 Arquivo ZIP criado: {zip_filename}")
print(f"📊 Tamanho: {os.path.getsize(zip_filename) / (1024**2):.2f} MB")

# Download
print("\n⬇️ Baixando resultados...")
files.download(zip_filename)

## 📊 Análise Detalhada (Opcional)

In [ ]:
# Gráficos com matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Gráfico de emoções
emotions = summary['emotions_distribution']
if emotions:
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(emotions.keys(), emotions.values(), color='skyblue')
    ax.set_xlabel('Emoção')
    ax.set_ylabel('Frequência')
    ax.set_title('Distribuição de Emoções Detectadas')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Nenhuma emoção detectada")

In [ ]:
# Timeline de atividades
activities = summary['activities_timeline']
if activities:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    colors = {'walking': 'green', 'sitting': 'blue', 'gesturing': 'orange'}
    
    for i, activity in enumerate(activities[:20]):  # Primeiras 20
        start = activity['start']
        end = activity['end']
        label = activity['label']
        color = colors.get(label, 'gray')
        
        ax.barh(i, end - start, left=start, height=0.8, color=color, alpha=0.7)
        ax.text(start + (end - start) / 2, i, label, 
                ha='center', va='center', fontsize=8, color='white', weight='bold')
    
    ax.set_xlabel('Frame')
    ax.set_ylabel('Evento')
    ax.set_title('Timeline de Atividades (Primeiros 20 Eventos)')
    ax.set_yticks(range(min(len(activities), 20)))
    ax.set_yticklabels([f"Evento {i+1}" for i in range(min(len(activities), 20))])
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Total de {len(activities)} atividades detectadas")
else:
    print("⚠️ Nenhuma atividade detectada")

## 🎓 Informações Adicionais

### 🚀 Velocidade no Colab vs Local:
- **GPU T4**: ~10-20x mais rápido que CPU
- **Modelos DeepFace**: Carregam mais rápido
- **Sem travamentos**: Mais RAM disponível

### 📚 Recursos:
- [Documentação do Projeto](https://github.com/Mavsousa30/techchallengefase4)
- [README](https://github.com/Mavsousa30/techchallengefase4/blob/main/README.md)
- [Troubleshooting Colab](https://github.com/Mavsousa30/techchallengefase4/blob/main/notebooks/TROUBLESHOOTING_COLAB.md)

### 💡 Dicas:
1. Use vídeos curtos (< 2 min) para testes rápidos
2. Ative a GPU para melhor performance
3. O primeiro run pode demorar (download de modelos)
4. Runs subsequentes são muito mais rápidos